In [1]:
from typing import List, Callable, Tuple

In [2]:
def read_data(path):
    f = open(path,"r")
    return [line.strip().split(" ") for line in f.readlines()]

In [3]:
full_dataset = read_data("data.txt")
full_dataset

[['inp', 'w'],
 ['mul', 'x', '0'],
 ['add', 'x', 'z'],
 ['mod', 'x', '26'],
 ['div', 'z', '1'],
 ['add', 'x', '15'],
 ['eql', 'x', 'w'],
 ['eql', 'x', '0'],
 ['mul', 'y', '0'],
 ['add', 'y', '25'],
 ['mul', 'y', 'x'],
 ['add', 'y', '1'],
 ['mul', 'z', 'y'],
 ['mul', 'y', '0'],
 ['add', 'y', 'w'],
 ['add', 'y', '9'],
 ['mul', 'y', 'x'],
 ['add', 'z', 'y'],
 ['inp', 'w'],
 ['mul', 'x', '0'],
 ['add', 'x', 'z'],
 ['mod', 'x', '26'],
 ['div', 'z', '1'],
 ['add', 'x', '11'],
 ['eql', 'x', 'w'],
 ['eql', 'x', '0'],
 ['mul', 'y', '0'],
 ['add', 'y', '25'],
 ['mul', 'y', 'x'],
 ['add', 'y', '1'],
 ['mul', 'z', 'y'],
 ['mul', 'y', '0'],
 ['add', 'y', 'w'],
 ['add', 'y', '1'],
 ['mul', 'y', 'x'],
 ['add', 'z', 'y'],
 ['inp', 'w'],
 ['mul', 'x', '0'],
 ['add', 'x', 'z'],
 ['mod', 'x', '26'],
 ['div', 'z', '1'],
 ['add', 'x', '10'],
 ['eql', 'x', 'w'],
 ['eql', 'x', '0'],
 ['mul', 'y', '0'],
 ['add', 'y', '25'],
 ['mul', 'y', 'x'],
 ['add', 'y', '1'],
 ['mul', 'z', 'y'],
 ['mul', 'y', '0'],
 ['add

# Part 1

In [4]:
def get_var_index(var: str) -> int:
    if var == "w":
        return 0
    elif var == "x":
        return 1
    elif var == "y":
        return 2
    elif var == "z":
        return 3
    else:
        raise Exception("Unknown var " + str(var))

In [5]:
Action = Callable[[List[int], List[int]], List[int]]

In [6]:
def get_inp(index: int) -> Action:
    def inp(number: List[int], values: List[int], current: int):
        values[index] = number[current]
    return inp

In [7]:
def get_sum(i1: int, i2: int) -> Action:
    def add(number: List[int], values: List[int]):
        values[i1] += values[i2]
    return add

In [8]:
def get_sum_const(i1: int, val: int) -> Action:
    def add_const(number: List[int], values: List[int]):
        values[i1] += val
    return add_const

In [9]:
add1 = get_sum_const(0, 12)
add2 = get_sum_const(1, 25)
arr = [0, 0, 0 ,0]
add1([], arr)
add2([], arr)
assert arr == [12, 25, 0, 0]

In [10]:
add1.__name__

'add_const'

In [11]:
def get_mul(i1: int, i2: int) -> Action:
    def mul(number: List[int], values: List[int]):
        values[i1] *= values[i2]
    return mul

In [12]:
def get_mul_const(i1: int, val: int) -> Action:
    def mul_const(number: List[int], values: List[int]):
        values[i1] *= val
    return mul_const

In [13]:
def get_div(i1: int, i2: int) -> Action:
    def div(number: List[int], values: List[int]):
        values[i1] = values[i1] // values[i2]
    return div

In [14]:
def get_div_const(i1: int, val: int) -> Action:
    def div_const(number: List[int], values: List[int]):
        values[i1] = values[i1] // val
    return div_const

In [15]:
def get_mod(i1: int, i2: int) -> Action:
    def mod(number: List[int], values: List[int]):
        values[i1] = values[i1] % values[i2]
    return div

In [16]:
def get_mod_const(i1: int, val: int) -> Action:
    def mod_const(number: List[int], values: List[int]):
        values[i1] = values[i1] % val
    return mod_const

In [17]:
def get_eql(i1: int, i2: int) -> Action:
    def eql(number: List[int], values: List[int]):
        values[i1] = 0 if (values[i1] == values[i2]) else 1
    return eql

In [18]:
def get_eql_const(i1: int, val: int) -> Action:
    def eql_const(number: List[int], values: List[int]):
        values[i1] = 0 if (values[i1] == val) else 1
    return eql_const

In [19]:
def is_var(string: str) -> bool:
    return string in ["w", "x", "y", "z"]

In [20]:
def compile_action(row: List[str]) -> Action:
    name = row[0]
    i1 = get_var_index(row[1])
    if name == "inp":
        return get_inp(i1)
    elif name == "add":
        return get_sum(i1, get_var_index(row[2])) if is_var(row[2]) else get_sum_const(i1, int(row[2]))
    elif name == "mul":
        return get_mul(i1, get_var_index(row[2])) if is_var(row[2]) else get_mul_const(i1, int(row[2]))
    elif name == "div":
        return get_div(i1, get_var_index(row[2])) if is_var(row[2]) else get_div_const(i1, int(row[2]))
    elif name == "mod":
        return get_mod(i1, get_var_index(row[2])) if is_var(row[2]) else get_mod_const(i1, int(row[2]))
    elif name == "eql":
        return get_eql(i1, get_var_index(row[2])) if is_var(row[2]) else get_eql_const(i1, int(row[2]))

In [24]:
def regroup_actions(actions) -> List[List[Action]]:
    groups = []
    current_group = []
    for action in actions:
        if action.__name__ == "inp" and len(current_group) > 0:
            groups.append(current_group)
            current_group = []
        current_group.append(action)
    if len(current_group) > 0:
        groups.append(current_group)
    return groups

In [25]:
assert len(regroup_actions([compile_action(a) for a in full_dataset])) == 14

In [26]:
def serialize(index, groups, number, values):
    return (i, *values)

In [27]:
def solve_for_group(index, groups, number, values, cache, reverse = True):
    ser = (index, *values)
    if ser in cache:
        return cache[ser]
    if index == 14:
        res = [x for x in number] if  values[3] == 0 else None
        cache[ser] = res
        return res

    # Logs the current work
    if (index == 2):
        print(number)
    
    for k in (range(9, 0, -1) if reverse else range(1, 10)):
        current_values = [v for v in values]
        number[index] = k
        actions = groups[index]
        # Apply the input action first
        actions[0](number, current_values, index)
        # Apply other actions
        apply_actions(number, current_values, actions[1:])
        result = solve_for_group(index + 1, groups, number, current_values, cache, reverse)
        if result is not None:
            cache[ser] = result
            return result
    cache[ser] = None
        

In [28]:
def find_biggest_value(dataset):
    action_groups = regroup_actions([compile_action(a) for a in dataset])
    number = [9] * 14
    solution = solve_for_group(0, action_groups, number, [0, 0, 0, 0], {})
    return "".join([str(x) for x in solution])

In [29]:
find_biggest_value(full_dataset)

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
[9, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[7, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[7, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[7, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[7, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[7, 4, 1, 1

'29991993698469'

# Part 2

In [31]:
def find_smallest_value(dataset):
    action_groups = regroup_actions([compile_action(a) for a in dataset])
    number = [1] * 14
    solution = solve_for_group(0, action_groups, number, [0, 0, 0, 0], {}, reverse=False)
    return "".join([str(x) for x in solution])

In [32]:
find_smallest_value(full_dataset)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 2, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
[1, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
[1, 4, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]


'14691271141118'